<a href="https://colab.research.google.com/github/jaksonvelasquez/cun/blob/main/Consulta_secop_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.6 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import requests
from google.colab import files

# Dictionary of parameters (using underscores for keys)
param_options = {
    'nombre_entidad': 'text',
    'nit_entidad': 'text',
    'departamento': 'text',
    'ciudad': 'text',
    'localizacion': 'text',
    'orden': 'text',
    'descripcion_del_proceso': 'text',
    'tipo_de_contrato': 'text',
    'modalidad_de_contratacion': 'text',
    'fecha_de_firma': 'date',
    'fecha_de_inicio_del_contrato': 'date',
    'fecha_de_fin_del_contrato': 'date',
    'fecha_de_inicio_de_ejecucion': 'date',
    'fecha_de_fin_de_ejecucion': 'date',
    'documento_proveedor': 'text',
    'proveedor_adjudicado': 'text',
    'valor_del_contrato': 'number',
    'nombre_representante_legal': 'text',
    'objeto_del_contrato': 'text'
}

# Function to fetch and download data
def fetch_data(query_params):
    base_url = 'https://www.datos.gov.co/resource/jbjy-vk9h.json'
    response = requests.get(base_url, params={"$where": " AND ".join(query_params.values())})
    df = pd.DataFrame(response.json())
    file_name = 'data.xlsx'
    df.to_excel(file_name, index=False)
    files.download(file_name)

# Event handler for the submit button
def handle_submit(b):
    query_params = {}
    # Access text input widget using key without spaces
    for param in chosen_params:
        param_name = param.value.replace(" ", "_")  # Replace spaces with underscores
        input_widget = text_inputs[param_name]
        value = input_widget.value
        if param_options[param_name] == 'date' and value:
            value = pd.to_datetime(value).strftime('%Y-%m-%d')
        if value:
            query_params[param_name] = f"{param_name}='{value}'"
    fetch_data(query_params)


# Function to update widgets based on number of parameters selected
def update_params(change):
    clear_output(wait=True)
    num = num_params.value
    global chosen_params, text_inputs
    chosen_params = [widgets.Dropdown(options=list(param_options.keys()), description=f'Parameter {i+1}:') for i in range(num)]
    text_inputs = {}
    for param in chosen_params:
        param.observe(update_text_inputs, names='value')
        text_inputs[param.value] = widgets.Text(description=f'Enter {param.value}:')
    box = widgets.VBox(chosen_params + list(text_inputs.values()) + [submit_button])
    display(box)

# Update text input labels based on dropdown selection
def update_text_inputs(change):
    old_value = change.old
    new_value = change.new
    # Update the dictionary to reflect the new parameter selection
    text_inputs[new_value] = text_inputs.pop(old_value)
    text_inputs[new_value].description = f'Enter {new_value}:'
    # Refresh display to show updated text inputs
    box = widgets.VBox(chosen_params + list(text_inputs.values()) + [submit_button])
    display(box)

# Initialize the UI
num_params.observe(update_params, 'value')
display(num_params)


# User selects the number of parameters
num_params = widgets.IntSlider(value=1, min=1, max=4, step=1, description='Number of parameters:')
submit_button = widgets.Button(description="Submit")
submit_button.on_click(handle_submit)



In [34]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import requests
from google.colab import files

# Definir constantes para parámetros
PARAM_OPTIONS = {
    'nombre_entidad': 'text', 'nit_entidad': 'text', 'departamento': 'text', 'ciudad': 'text',
    'localizacion': 'text', 'orden': 'text', 'descripcion_del_proceso': 'text', 'tipo_de_contrato': 'text',
    'modalidad_de_contratacion': 'text', 'fecha_de_firma': 'date', 'fecha_de_inicio_del_contrato': 'date',
    'fecha_de_fin_del_contrato': 'date', 'fecha_de_inicio_de_ejecucion': 'date', 'fecha_de_fin_de_ejecucion': 'date',
    'documento_proveedor': 'text', 'proveedor_adjudicado': 'text', 'valor_del_contrato': 'number',
    'nombre_representante_legal': 'text', 'objeto_del_contrato': 'text'
}

# Función para inicializar la UI
def initialize_ui():
    global num_params, submit_button, chosen_params, text_inputs
    num_params = widgets.IntSlider(value=1, min=1, max=len(PARAM_OPTIONS), step=1, description='Number of parameters:')
    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(handle_submit)
    chosen_params = []
    text_inputs = {}
    num_params.observe(update_params, 'value')
    display(num_params)

# Función para manejar la presentación de los widgets
def display_widgets():
    widget_list = [num_params] + chosen_params + list(text_inputs.values()) + [submit_button]
    box = widgets.VBox(widget_list)
    display(box)

# Función para actualizar los widgets basado en el número de parámetros seleccionados
def update_params(change):
    clear_output(wait=True)
    chosen_params.clear()
    text_inputs.clear()
    for i in range(num_params.value):
        param_dropdown = widgets.Dropdown(options=list(PARAM_OPTIONS.keys()), description=f'Parameter {i+1}:')
        param_dropdown.observe(update_text_input, names='value')
        chosen_params.append(param_dropdown)
        # Create text inputs with a unique key to prevent overwriting
        text_inputs[param_dropdown.description] = widgets.Text(description='Enter value:')
    display_widgets()

# Función para actualizar las entradas de texto basadas en la selección del dropdown
def update_text_input(change):
    param_description = change.owner.description
    new_key = f"{param_description} - {change.new}"
    old_value = text_inputs.pop(param_description, None)
    new_value = old_value.value if old_value else ''
    text_inputs[new_key] = widgets.Text(value=new_value, description=f'Enter {change.new}:')
    display_widgets()

# Función para manejar el evento de envío
def handle_submit(b):
    query_params = build_query_params()
    fetch_data(query_params)

# Función para construir los parámetros de consulta
def build_query_params():
    query_params = {}
    for key, widget in text_inputs.items():
        param_name = key.split('-')[-1].strip().replace(" ", "_")
        value = widget.value
        if PARAM_OPTIONS[param_name] == 'date' and value:
            value = pd.to_datetime(value).strftime('%Y-%m-%d')
        if value:
            query_params[param_name] = f"{param_name}='{value}'"
    return query_params

# Función para obtener y descargar datos
def fetch_data(query_params):
    base_url = 'https://www.datos.gov.co/resource/jbjy-vk9h.json'
    response = requests.get(base_url, params={"$where": " AND ".join(query_params.values())})
    df = pd.DataFrame(response.json())
    file_name = 'data.xlsx'
    df.to_excel(file_name, index=False)
    files.download(file_name)

initialize_ui()

KeyError: 'Parameter_1:'

In [37]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import requests
from google.colab import files

# Definir constantes para parámetros
PARAM_OPTIONS = {
    'nombre_entidad': 'text', 'nit_entidad': 'text', 'departamento': 'text', 'ciudad': 'text',
    'localizacion': 'text', 'orden': 'text', 'descripcion_del_proceso': 'text', 'tipo_de_contrato': 'text',
    'modalidad_de_contratacion': 'text', 'fecha_de_firma': 'date', 'fecha_de_inicio_del_contrato': 'date',
    'fecha_de_fin_del_contrato': 'date', 'fecha_de_inicio_de_ejecucion': 'date', 'fecha_de_fin_de_ejecucion': 'date',
    'documento_proveedor': 'text', 'proveedor_adjudicado': 'text', 'valor_del_contrato': 'number',
    'nombre_representante_legal': 'text', 'objeto_del_contrato': 'text'
}

# Función para inicializar la UI
def initialize_ui():
    global num_params, submit_button, chosen_params, text_inputs
    num_params = widgets.IntSlider(value=1, min=1, max=len(PARAM_OPTIONS), step=1, description='Number of parameters:')
    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(handle_submit)
    chosen_params = []
    text_inputs = {}
    num_params.observe(update_params, 'value')
    display(num_params)

# Función para manejar la presentación de los widgets
def display_widgets():
    widget_list = [num_params] + [widgets.HBox([param, text_inputs[param.description]]) for param in chosen_params] + [submit_button]
    box = widgets.VBox(widget_list)
    display(box)

# Función para actualizar los widgets basado en el número de parámetros seleccionados
def update_params(change):
    clear_output(wait=True)
    chosen_params.clear()
    text_inputs.clear()
    for i in range(num_params.value):
        param_dropdown = widgets.Dropdown(options=list(PARAM_OPTIONS.keys()), description=f'Parameter {i+1}:')
        param_dropdown.observe(update_text_input, names='value')
        chosen_params.append(param_dropdown)
        text_inputs[param_dropdown.description] = widgets.Text(description='Enter value:')
    display_widgets()

# Función para actualizar las entradas de texto basadas en la selección del dropdown
def update_text_input(change):
    param_dropdown = change.owner
    text_input = text_inputs[param_dropdown.description]
    text_input.description = f'Enter {change.new}:'

# Función para manejar el evento de envío
def handle_submit(b):
    query_params = build_query_params()
    fetch_data(query_params)

# Función para construir los parámetros de consulta
def build_query_params():
    query_params = {}
    for param_dropdown, text_input in zip(chosen_params, text_inputs.values()):
        param_name = param_dropdown.value.replace(" ", "_")
        value = text_input.value
        if PARAM_OPTIONS[param_name] == 'date' and value:
            value = pd.to_datetime(value).strftime('%Y-%m-%d')
        if value:
            query_params[param_name] = f"{param_name}='{value}'"
    return query_params

# Función para obtener y descargar datos
def fetch_data(query_params):
    base_url = 'https://www.datos.gov.co/resource/jbjy-vk9h.csv'
    response = requests.get(base_url, params={"$where": " AND ".join(query_params.values())})
    df = pd.DataFrame(response.csv())
    file_name = 'data.xlsx'
    df.to_excel(file_name, index=False)
    files.download(file_name)

initialize_ui()

AttributeError: 'Response' object has no attribute 'csv'

In [38]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import requests
from google.colab import files

# Definir constantes para parámetros
PARAM_OPTIONS = {
    'nombre_entidad': 'text', 'nit_entidad': 'text', 'departamento': 'text', 'ciudad': 'text',
    'localizacion': 'text', 'orden': 'text', 'descripcion_del_proceso': 'text', 'tipo_de_contrato': 'text',
    'modalidad_de_contratacion': 'text', 'fecha_de_firma': 'date', 'fecha_de_inicio_del_contrato': 'date',
    'fecha_de_fin_del_contrato': 'date', 'fecha_de_inicio_de_ejecucion': 'date', 'fecha_de_fin_de_ejecucion': 'date',
    'documento_proveedor': 'text', 'proveedor_adjudicado': 'text', 'valor_del_contrato': 'number',
    'nombre_representante_legal': 'text', 'objeto_del_contrato': 'text'
}

# Función para inicializar la UI
def initialize_ui():
    global num_params, submit_button, chosen_params, text_inputs
    num_params = widgets.IntSlider(value=1, min=1, max=len(PARAM_OPTIONS), step=1, description='Number of parameters:')
    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(handle_submit)
    chosen_params = []
    text_inputs = {}
    num_params.observe(update_params, 'value')
    display(num_params)

# Función para manejar la presentación de los widgets
def display_widgets():
    widget_list = [num_params] + [widgets.HBox([param, text_inputs[param.description]]) for param in chosen_params] + [submit_button]
    box = widgets.VBox(widget_list)
    display(box)

# Función para actualizar los widgets basado en el número de parámetros seleccionados
def update_params(change):
    clear_output(wait=True)
    chosen_params.clear()
    text_inputs.clear()
    for i in range(num_params.value):
        param_dropdown = widgets.Dropdown(options=list(PARAM_OPTIONS.keys()), description=f'Parameter {i+1}:')
        param_dropdown.observe(update_text_input, names='value')
        chosen_params.append(param_dropdown)
        text_inputs[param_dropdown.description] = widgets.Text(description='Enter value:')
    display_widgets()

# Función para actualizar las entradas de texto basadas en la selección del dropdown
def update_text_input(change):
    param_dropdown = change.owner
    text_input = text_inputs[param_dropdown.description]
    text_input.description = f'Enter {change.new}:'

# Función para manejar el evento de envío
def handle_submit(b):
    query_params = build_query_params()
    fetch_data(query_params)

# Función para construir los parámetros de consulta
def build_query_params():
    query_params = {}
    for param_dropdown, text_input in zip(chosen_params, text_inputs.values()):
        param_name = param_dropdown.value.replace(" ", "_")
        value = text_input.value
        if PARAM_OPTIONS[param_name] == 'date' and value:
            value = pd.to_datetime(value).strftime('%Y-%m-%d')
        if value:
            query_params[param_name] = value
    return query_params

# Función para obtener y descargar datos
def fetch_data(query_params):
    base_url = 'https://www.datos.gov.co/resource/jbjy-vk9h.csv'
    response = requests.get(base_url, params=query_params)
    if response.ok:
        file_name = 'data.csv'
        with open(file_name, 'wb') as file:
            file.write(response.content)
        files.download(file_name)
    else:
        print("Failed to fetch data:", response.status_code)

initialize_ui()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import requests
from google.colab import files

# Definir constantes para parámetros
PARAM_OPTIONS = {
    'nombre_entidad': 'text', 'nit_entidad': 'text', 'departamento': 'text', 'ciudad': 'text',
    'localizacion': 'text', 'orden': 'text', 'descripcion_del_proceso': 'text', 'tipo_de_contrato': 'text',
    'modalidad_de_contratacion': 'text', 'fecha_de_firma': 'date', 'fecha_de_inicio_del_contrato': 'date',
    'fecha_de_fin_del_contrato': 'date', 'fecha_de_inicio_de_ejecucion': 'date', 'fecha_de_fin_de_ejecucion': 'date',
    'documento_proveedor': 'text', 'proveedor_adjudicado': 'text', 'valor_del_contrato': 'number',
    'nombre_representante_legal': 'text', 'objeto_del_contrato': 'text'
}

# Función para inicializar la UI
def initialize_ui():
    global num_params, submit_button, chosen_params, text_inputs
    num_params = widgets.IntSlider(value=1, min=1, max=len(PARAM_OPTIONS), step=1, description='Number of parameters:')
    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(handle_submit)
    chosen_params = []
    text_inputs = {}
    num_params.observe(update_params, 'value')
    display(num_params)

# Función para manejar la presentación de los widgets
def display_widgets():
    widget_list = [num_params] + [widgets.HBox([param, text_inputs[param.description]]) for param in chosen_params] + [submit_button]
    box = widgets.VBox(widget_list)
    display(box)

# Función para actualizar los widgets basado en el número de parámetros seleccionados
def update_params(change):
    clear_output(wait=True)
    chosen_params.clear()
    text_inputs.clear()
    for i in range(num_params.value):
        param_dropdown = widgets.Dropdown(options=list(PARAM_OPTIONS.keys()), description=f'Parameter {i+1}:')
        param_dropdown.observe(update_text_input, names='value')
        chosen_params.append(param_dropdown)
        text_inputs[param_dropdown.description] = widgets.Text(description='Enter value:')
    display_widgets()

# Función para actualizar las entradas de texto basadas en la selección del dropdown
def update_text_input(change):
    param_dropdown = change.owner
    text_input = text_inputs[param_dropdown.description]
    text_input.description = f'Enter {change.new}:'

# Función para manejar el evento de envío
def handle_submit(b):
    query_params = build_query_params()
    fetch_data(query_params)

# Función para construir los parámetros de consulta
def build_query_params():
    query_parts = []
    for param_dropdown, text_input in zip(chosen_params, text_inputs.values()):
        param_name = param_dropdown.value.replace(" ", "_")
        value = text_input.value
        if PARAM_OPTIONS[param_name] == 'date' and value:
            value = pd.to_datetime(value).strftime('%Y-%m-%d')
        if value:
            query_parts.append(f"{param_name}='{value}'")
    query_string = " AND ".join(query_parts)
    return {"$where": query_string}

# Función para obtener y descargar datos
def fetch_data(query_params):
    base_url = 'https://www.datos.gov.co/resource/jbjy-vk9h.csv'
    response = requests.get(base_url, params=query_params)
    if response.ok:
        file_name = 'data.csv'
        with open(file_name, 'wb') as file:
            file.write(response.content)
        files.download(file_name)
    else:
        print("Failed to fetch data:", response.status_code)

initialize_ui()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import requests
from google.colab import files
import  io

# Definir constantes para parámetros
PARAM_OPTIONS = {
    'nombre_entidad': 'text', 'nit_entidad': 'text', 'departamento': 'text', 'ciudad': 'text',
    'localizacion': 'text', 'orden': 'text', 'descripcion_del_proceso': 'text', 'tipo_de_contrato': 'text',
    'modalidad_de_contratacion': 'text', 'fecha_de_firma': 'date', 'fecha_de_inicio_del_contrato': 'date',
    'fecha_de_fin_del_contrato': 'date', 'fecha_de_inicio_de_ejecucion': 'date', 'fecha_de_fin_de_ejecucion': 'date',
    'documento_proveedor': 'text', 'proveedor_adjudicado': 'text', 'valor_del_contrato': 'number',
    'nombre_representante_legal': 'text', 'objeto_del_contrato': 'text'
}

# Función para inicializar la UI
def initialize_ui():
    global num_params, submit_button, chosen_params, text_inputs
    num_params = widgets.IntSlider(value=1, min=1, max=len(PARAM_OPTIONS), step=1, description='Number of parameters:')
    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(handle_submit)
    chosen_params = []
    text_inputs = {}
    num_params.observe(update_params, 'value')
    display(num_params)

# Función para manejar la presentación de los widgets
def display_widgets():
    widget_list = [num_params] + [widgets.HBox([param, text_inputs[param.description]]) for param in chosen_params] + [submit_button]
    box = widgets.VBox(widget_list)
    display(box)

# Función para actualizar los widgets basado en el número de parámetros seleccionados
def update_params(change):
    clear_output(wait=True)
    chosen_params.clear()
    text_inputs.clear()
    for i in range(num_params.value):
        param_dropdown = widgets.Dropdown(options=list(PARAM_OPTIONS.keys()), description=f'Parameter {i+1}:')
        param_dropdown.observe(update_text_input, names='value')
        chosen_params.append(param_dropdown)
        text_inputs[param_dropdown.description] = widgets.Text(description='Enter value:')
    display_widgets()

# Función para actualizar las entradas de texto basadas en la selección del dropdown
def update_text_input(change):
    param_dropdown = change.owner
    text_input = text_inputs[param_dropdown.description]
    text_input.description = f'Enter {change.new}:'

# Función para manejar el evento de envío
def handle_submit(b):
    query_params = build_query_params()
    fetch_data(query_params)

# Función para construir los parámetros de consulta
def build_query_params():
    query_parts = []
    for param_dropdown, text_input in zip(chosen_params, text_inputs.values()):
        param_name = param_dropdown.value.replace(" ", "_")
        value = text_input.value.strip()
        if PARAM_OPTIONS[param_name] == 'date' and value:
            value = pd.to_datetime(value).strftime('%Y-%m-%d')
        if PARAM_OPTIONS[param_name] == 'text' and value:
            query_parts.append(f"lower({param_name}) like '%{value.lower()}%'")
        elif value:
            query_parts.append(f"{param_name}='{value}'")
    query_string = " AND ".join(query_parts)
    return {"$where": query_string}

# Función para obtener y descargar datos en formato Excel
def fetch_data(query_params):
    base_url = 'https://www.datos.gov.co/resource/jbjy-vk9h.csv'  # Obtenemos los datos en formato CSV
    response = requests.get(base_url, params=query_params)
    if response.ok:
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))  # Convertimos de CSV a DataFrame
        file_name = 'data.xlsx'
        df.to_excel(file_name, index=False)  # Guardamos el DataFrame como Excel
        files.download(file_name)  # Descargamos el archivo Excel
    else:
        print("Failed to fetch data:", response.status_code)

initialize_ui()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import requests
from google.colab import files

# Definir constantes para parámetros
PARAM_OPTIONS = {
    'nombre_entidad': 'text', 'nit_entidad': 'text', 'departamento': 'text', 'ciudad': 'text',
    'localizacion': 'text', 'orden': 'text', 'descripcion_del_proceso': 'text', 'tipo_de_contrato': 'text',
    'modalidad_de_contratacion': 'text', 'fecha_de_firma': 'date', 'fecha_de_inicio_del_contrato': 'date',
    'fecha_de_fin_del_contrato': 'date', 'fecha_de_inicio_de_ejecucion': 'date', 'fecha_de_fin_de_ejecucion': 'date',
    'documento_proveedor': 'text', 'proveedor_adjudicado': 'text', 'valor_del_contrato': 'number',
    'nombre_representante_legal': 'text', 'objeto_del_contrato': 'text'
}

# Función para inicializar la UI
def initialize_ui():
    global num_params, submit_button, chosen_params, text_inputs
    num_params = widgets.IntSlider(value=1, min=1, max=len(PARAM_OPTIONS), step=1, description='Number of parameters:')
    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(handle_submit)
    chosen_params = []
    text_inputs = {}
    num_params.observe(update_params, 'value')
    display(num_params)

# Función para manejar la presentación de los widgets
def display_widgets():
    widget_list = [num_params] + [widgets.HBox([param, text_inputs[param.description]]) for param in chosen_params] + [submit_button]
    box = widgets.VBox(widget_list)
    display(box)

# Función para actualizar los widgets basado en el número de parámetros seleccionados
def update_params(change):
    clear_output(wait=True)
    chosen_params.clear()
    text_inputs.clear()
    for i in range(num_params.value):
        param_dropdown = widgets.Dropdown(options=list(PARAM_OPTIONS.keys()), description=f'Parameter {i+1}:')
        param_dropdown.observe(update_text_input, names='value')
        chosen_params.append(param_dropdown)
        text_inputs[param_dropdown.description] = widgets.Text(description='Enter value:')
    display_widgets()

# Función para actualizar las entradas de texto basadas en la selección del dropdown
def update_text_input(change):
    param_dropdown = change.owner
    text_input = text_inputs[param_dropdown.description]
    text_input.description = f'Enter {change.new}:'

# Función para manejar el evento de envío
def handle_submit(b):
    query_params = build_query_params()
    fetch_data(query_params)

# Función para construir los parámetros de consulta
def build_query_params():
    query_parts = []
    for param_dropdown, text_input in zip(chosen_params, text_inputs.values()):
        param_name = param_dropdown.value.replace(" ", "_")
        value = text_input.value.strip()
        if PARAM_OPTIONS[param_name] == 'date' and value:
            value = pd.to_datetime(value).strftime('%Y-%m-%d')
        if PARAM_OPTIONS[param_name] == 'text' and value:
            query_parts.append(f"lower({param_name}) like '%{value.lower()}%'")
        elif value:
            query_parts.append(f"{param_name}='{value}'")
    query_string = " AND ".join(query_parts)
    return {"$where": query_string}

# Función para obtener y descargar datos en formato Excel
def fetch_data(query_params):
    base_url = 'https://www.datos.gov.co/resource/jbjy-vk9h.csv'  # Obtenemos en formato CSV primero
    response = requests.get(base_url, params=query_params)
    if response.ok:
        df = pd.read_csv(pd.compat.StringIO(response.text))  # Convertimos de CSV a DataFrame
        file_name = 'data.xlsx'
        df.to_excel(file_name, index=False)  # Guardamos DataFrame como Excel
        files.download(file_name)  # Descargamos el archivo Excel
    else:
        print("Failed to fetch data:", response.status_code)

initialize_ui()

AttributeError: module 'pandas.compat' has no attribute 'StringIO'